In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end


using MorphoMol
using JLD2
using LinearAlgebra
using Rotations
using Printf

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [2]:
function get_aggregate_simulation_evaluation(folder)
    mol_type = "6r7m"
    assembly_successes = 0
    total_simulations = 0
    id_index = 1
    steps_from_init_to_success = Vector{Int}([])
    for file in readdir(folder)
        if split(file, ".")[end] == "jld2"
            try
                @load "$folder$file" input output
                exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
                exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
                theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])

                total_simulations += 1
                if theta < 2.0
                    assembly_successes += 1
                    push!(steps_from_init_to_success, argmin(output["Es"]))
                end
            catch e
                println("Error in $file")
            end
        end
    end
    return total_simulations, assembly_successes/total_simulations, sum(steps_from_init_to_success)/length(steps_from_init_to_success)
end

get_aggregate_simulation_evaluation (generic function with 1 method)

In [3]:
folder = "../../Data/collected_simulation_results/rwm_ma_2_6r7m/"
ts, sr, sts = get_aggregate_simulation_evaluation(folder)
println("Moprphometric Approach")
@printf("Number of simulations: %.0f, Success ratio: %.3f, Steps to success: %.0f\n", ts, sr, sts)

Moprphometric Approach
Number of simulations: 198, Success ratio: 0.056, Steps to success: 5002


In [4]:
folder = "../../Data/collected_simulation_results/rwm_wp_2_6r7m/"
ts, sr, sts = get_aggregate_simulation_evaluation(folder)
println("Moprphometric Approach with Persistence")
@printf("Number of simulations: %.0f, Success ratio: %.3f, Steps to success: %.0f\n", ts, sr, sts)

Moprphometric Approach with Persistence
Number of simulations: 198, Success ratio: 0.121, Steps to success: 542


In [5]:
folder = "../../Data/collected_simulation_results/rwm_op_2_6r7m/"
ts, sr, sts = get_aggregate_simulation_evaluation(folder)
println("Only Persistence")
@printf("Number of simulations: %.0f, Success ratio: %.3f, Steps to success: %.0f\n", ts, sr, sts)

Only Persistence
Number of simulations: 76, Success ratio: 0.039, Steps to success: 1018
